In [2]:
import os
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

# Function to load and preprocess images
def loading_preprossing(path):
    img_list = []
    for img_path in os.listdir(path):
        img = cv2.imread(os.path.join(path, img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (255, 255))
        img = img / 255.0
        img_list.append(img)
    return img_list

# Load the datasets
normal_images = loading_preprossing('/kaggle/input/chest-xray-pneumonia/chest_xray/train/NORMAL')
pneumonia_images = loading_preprossing('/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA')

# Randomly sample the specified number of images from each class
normal_samples = random.sample(normal_images, 1340)
pneumonia_samples = random.sample(pneumonia_images, 2000)

# Combine the datasets
train_images = normal_samples + pneumonia_samples
train_labels = [0] * len(normal_samples) + [1] * len(pneumonia_samples)

# Convert to numpy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Convert labels to categorical (one-hot encoding)
y_train = np.eye(2)[y_train]
y_test = np.eye(2)[y_test]

# Define the model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(255, 255, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Save the model
model.save('/kaggle/working/chest_xray_model.h5')

# Load the model
loaded_model = load_model('/kaggle/working/chest_xray_model.h5')

# Evaluate the loaded model on test data
test_loss, test_acc = loaded_model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')


2024-06-24 22:45:59.797573: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 22:45:59.797748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 22:45:59.956029: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 253, 253, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 126, 126, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 124, 124, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,087,426 (49.92 MB)

 Trainable params: 13,087,426 (49.92 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 234s 3s/step - accuracy: 0.6807 - loss: 0.6464 - val_accuracy: 0.9566 - val_loss: 0.1562
Epoch 2/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 252s 3s/step - accuracy: 0.9457 - loss: 0.1483 - val_accuracy: 0.9506 - val_loss: 0.1154
Epoch 3/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 261s 3s/step - accuracy: 0.9724 - loss: 0.0945 - val_accuracy: 0.9446 - val_loss: 0.1364
Epoch 4/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 220s 3s/step - accuracy: 0.9700 - loss: 0.0806 - val_accuracy: 0.9521 - val_loss: 0.1617
Epoch 5/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 222s 3s/step - accuracy: 0.9729 - loss: 0.0687 - val_accuracy: 0.9596 - val_loss: 0.1280
Epoch 6/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 221s 3s/step - accuracy: 0.9613 - loss: 0.0919 - val_accuracy: 0.9656 - val_loss: 0.1156
Epoch 7/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.9862 - loss: 0.0362 - val_accuracy: 0.9641 - val_loss: 0.0863
Epoch 8/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 262s 3s/step - accuracy: 0.9893 - loss: 0.0298 - val_accuracy: 0.9551 - v

In [5]:
test_class1=loading_preprossing('/kaggle/input/chest-xray-pneumonia/chest_xray/test/NORMAL')
test_class2=loading_preprossing('/kaggle/input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA')

In [10]:
len(test_class1)

234

In [6]:
test_list=test_class1+test_class2
test_label=[0]*len(test_class1)+[1]*len(test_class2)
print('the label lenght: ',len(test_label))
print('the train list: ',len(test_list))

the label lenght:  624
the train list:  624


In [15]:
test_label

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [8]:
x_test=np.array(test_list)
Preds=model.predict(x_test)


20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 752ms/step


In [14]:
preds=np.argmax(np.round(Preds),axis=1)
preds

array([1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [17]:
import pandas as pd
df=pd.DataFrame({'real':test_label,'predicted':preds})

In [18]:
df

,real,predicted
0,0,1
1,0,1
2,0,0
3,0,1
4,0,0
...,...,...
619,1,1
620,1,1
621,1,1
622,1,1


In [19]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [20]:
# Calculate accuracy
accuracy = accuracy_score(test_label,preds)
print(f'Accuracy: {accuracy}')

# Generate a classification report
class_report = classification_report(test_label,preds, target_names=['Normal', 'Pneumonia'])
print('Classification Report:\n', class_report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(test_label,preds)
print('Confusion Matrix:\n', conf_matrix)

Accuracy: 0.7852564102564102
Classification Report:
               precision    recall  f1-score   support

      Normal       0.96      0.44      0.61       234
   Pneumonia       0.75      0.99      0.85       390

    accuracy                           0.79       624
   macro avg       0.86      0.72      0.73       624
weighted avg       0.83      0.79      0.76       624

Confusion Matrix:
 [[104 130]
 [  4 386]]


In [21]:
model.save('classification disease.h5')
print('save')

save
